In [ ]:
import numpy as np
import pandas as pd
    
import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub

import nltk
from nltk.corpus import stopwords

import emoji
import re
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler

import matplotlib.pyplot as plt
import sqlite3
import wordcloud
import seaborn as sns


In [ ]:
nltk.download('stopwords')

In [ ]:
DATA_TABLES= ['sms','yt','my-collection','spam-word','emoji']
MAX_LENGTH =  50
MAX_CURRENCY_FLAG = 2
MAX_SPAM_WORDS = 1
MAX_EMOJI = 2
MAX_CONATANS = 1
MAX_EMAIL= 1
MAX_PHONE = 1

In [ ]:
con = sqlite3.connect(r"cyradar_model/dataset.sqlite")

temp = []
for i in range(len(DATA_TABLES)):
    temp.append(pd.read_sql(f"SELECT * FROM '{DATA_TABLES[i]}'",con))
con.close()

df = pd.concat(temp,ignore_index=True)


In [ ]:
sns.countplot(x="category",data=df)

In [ ]:
data_ham  = df[df['category'] == 0].copy()
data_spam = df[df['category'] == 1].copy()

def show_wordcloud(df, title):
    text = ' '.join(df['comment'].astype(str).tolist())
    stopwords = set(wordcloud.STOPWORDS)
    
    fig_wordcloud = wordcloud.WordCloud(stopwords=stopwords,background_color='lightgrey',
                    colormap='viridis', width=800, height=600).generate(text)
    
    plt.figure(figsize=(10,7), frameon=True)
    plt.imshow(fig_wordcloud)  
    plt.axis('off')
    plt.title(title, fontsize=20 )
    plt.show()
show_wordcloud(data_spam, "Spam messages")
show_wordcloud(data_ham, "Ham messages")

In [ ]:
class UpSample:
    def fit(self, x, y=None):
        return self

    def transform(self, df):
        ros = RandomOverSampler(random_state=73133)

        x, y = ros.fit_resample(
            df[["comment","currency","length","spam_word","emoji","contain","email","phone"]].values
            ,df['category'].values
        )
        df =  pd.DataFrame(x, columns=["comment","currency","length","spam_word","emoji","contain","email","phone"])
        df['category'] = y
        return  df

class ConvertData:
    def fit(self, x, y=None):
        return self

    def transform(self, df):
        df = df.drop_duplicates()
        df = df.dropna()
        df["category"] = df["category"].astype(bool)
        df["comment"] = df["comment"].astype(str)
        return df


class RemoveStopWordsPunctuation:
    def fit(self, x, y=None):
        return self

    def __remove_punctuation_stopwords(self, text):
        pattern = re.compile("[{}]".format(re.escape("!\"#&'()*,-/:;<=>?[\\]^_`{|}~")))
        text = " ".join(
            [
                word.strip()
                for word in pattern.sub(" ", text.lower()).split()
                if word not in set(stopwords.words("english"))
            ]
        )
        return text

    def transform(self, df):
        df["comment"] = df["comment"].apply(self.__remove_punctuation_stopwords)
        return df


class AddLengthFlag:
    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X["length"] = X["comment"].str.len().astype(np.float32) / MAX_LENGTH
        return X


class AddCurrencyFlag:
    def __init__(self) -> None:
        self.currency_symbols = ["₤", "₨", "€", "₹", "₿", "$"]
        self.pattern = "([\$₤₨€₹₿]+ *[0-9]* *[\.,]?[0-9]*)|([0-9]* *[\.,]?[0-9]* *[\$₤₨€₹₿]+)"

    def fit(self, x, y=None):
        return self

    def __add_currency_count(self, text):
        return len(re.findall(self.pattern, text)) / MAX_CURRENCY_FLAG

    def transform(self, df):
        df["currency"] = df["comment"].apply(self.__add_currency_count).astype(np.float32)
        return df


class AddSpamWordsFlag:
    def __init__(self) -> None:
        self.spam_words = [
            "urgent",
            "exclusive",
            "limited time",
            "free",
            "guaranteed",
            "act now",
            "discount",
            "special offer",
            "prize",
            "instant",
            "cash",
            "save",
            "win",
            "best",
            "secret",
            "incredible",
            "congratulations",
            "approved",
            "risk free",
            "hidden",
            "bonus",
            "sale",
            "amazing",
            "extra cash",
            "opportunity",
            "easy",
            "double your",
            "best price",
            "cash back",
            "deal",
            "earn",
            "money",
            "no obligation",
            "profit",
            "results",
            "exciting",
            "unbelievable",
            "jackpot",
            "fantastic",
            "instant access",
            "million dollars",
            "discounted",
            "last chance",
            "exclusive offer",
            "big savings",
            "limited offer",
            "free trial",
            "special promotion",
            "secret revealed",
            "valuable",
            "money-back guarantee",
            "lowest price",
            "save money",
            "make money",
            "no risk",
            "exclusive deal",
            "limited supply",
            "huge",
            "incredible offer",
            "prize winner",
            "earn extra income",
            "limited spots",
            "new offer",
            "best deal",
            "don't miss out",
            "great savings",
            "top offer",
            "double your income",
            "discount code",
            "fast cash",
            "top-rated",
            "best value",
            "no cost",
            "elite",
            "act fast",
            "unbeatable",
            "cash prize",
            "limited availability",
            "special discount",
            "quick cash",
            "no catch",
            "instant approval",
            "big discount",
            "easy money",
            "insider",
            "invitation",
            "free shipping",
            "huge discount",
            "extra income",
            "secret formula",
            "no strings attached",
            "money-making",
            "dream come true",
            "massive",
            "free gift",
            "incredible opportunity",
            "risk-free trial",
            "instant money",
            "special price",
            "no purchase necessary",
            "now",
        ]

    def fit(self, x, y=None):
        return self

    def __add_currency_count(self, text):
        return float(sum(text.count(symbol) for symbol in self.spam_words) / MAX_SPAM_WORDS)

    def transform(self, df):
        df["spam_word"] = df["comment"].apply(self.__add_currency_count).astype(np.float32)
        return df


class AddEmojiFlag:
    def __init__(self) -> None:
        self.emoji_symbols = "[💭|🔝|🆗|🎉|🎊|📯|🙌|😂|💸|👉|📢|🚀|💲|💣|🔱|💼|🆙|⏳|✨|💌|💎|🆕|🔞|💡|💰|👑|⭐|🌟|🎤|⚡|📈|💵|🏆|💪|🔓|🆓|🎰|⌚|🚨|💢|📮|🔥|🎈|🎥|🔔|💯|🎶|🔗|🎁|📚|🔊|👍|👏|📱|📝|🤑|🏅|🔒|📣|💥]"

    def fit(self, x, y=None):
        return self

    def __add_currency_count(self, text):
        return float(len(re.findall(self.emoji_symbols, text)) / MAX_EMOJI)

    def transform(self, df):
        df["emoji"] = df["comment"].apply(self.__add_currency_count).astype(np.float32)
        return df


class AddContainFlag:
    def fit(self, x, y=None):
        return self

    def __add_first_count(self, text):
        pattern = "[0-9]*%|T&C"
        return len(re.findall(pattern, text))

    def __add_second_count(self, text):
        pattern = "(https:\/\/www\.|http:\/\/www\.|https:\/\/|http:\/\/)?[a-zA-Z0-9]{2,}(\.[a-zA-Z0-9]{2,})(\.[a-zA-Z0-9]{2,})?"
        return len(re.findall(pattern, text))

    def transform(self, df):
        df["contain"] = df["comment"].apply(self.__add_first_count)
        df["contain"] = df["contain"] + df["comment"].apply(self.__add_second_count)
        df['contain'] = df['contain'].astype(np.float32) / MAX_CONATANS
        return df


class AddEmailFlag:
    def fit(self, x, y=None):
        return self

    def __add_email_count(self, text):
        pattern = "[\w]+@[\w]+\.\w+"
        return float(len(re.findall(pattern, text))  /MAX_EMAIL)

    def transform(self, df):
        df["email"] = df["comment"].apply(self.__add_email_count).astype(np.float32)
        return df


class AddPhoneFlag:
    def fit(self, x, y=None):
        return self

    def __add_phone_no_count(self, text):
        pattern = "\+?[0-9]?[0-9]? ?0?[0-9]{10}"
        return len(re.findall(pattern, text))

    def __add_phone_no_count_1(self, text):
        pattern = "\+?[0-9]?\d{3}[ -]?\d{3}[ -]?\d{4}"
        return len(re.findall(pattern, text))

    def transform(self, df):
        df["phone"] = df["comment"].apply(self.__add_phone_no_count)
        df["phone"] = df["phone"] + df["comment"].apply(self.__add_phone_no_count_1)
        df["phone"] = df["phone"].astype(np.float32) / MAX_PHONE


        return df


class RemovePhoneLinkEmail:
    def fit(self, x, y=None):
        return self

    def __remove(self, text):
        text = re.sub("\$[0-9]*([\.,][0-9]{2})*\$?", "", text)
        text = re.sub("\+?[0-9]?[0-9]? ?0?[0-9]{10}", "", text)
        text = re.sub("\+?[0-9]?\d{3}[ -]?\d{3}[ -]?\d{4}", "", text)
        text = re.sub(
            r"(https:\/\/www\.|http:\/\/www\.|https:\/\/|http:\/\/)?[a-zA-Z0-9]{2,}(\.[a-zA-Z0-9]{2,})(\.[a-zA-Z0-9]{2,})?",
            "",
            text,
        )
        text = re.sub(r"[\w]+@[\w]+\.\w+", "", text)
        text = emoji.replace_emoji(text)
        return text

    def transform(self, df):
        df["comment"] = df["comment"].apply(self.__remove)
        return df


class LemmatizeText:
    def __init__(self):
        self.lemmatizer = nltk.WordNetLemmatizer()

    def fit(self, X, y=None):
        return self

    def __lemmatize_text(self, text):
        return " ".join(
            [self.lemmatizer.lemmatize(word) for word in re.split("\W+", text)]
        ).strip()

    def transform(self, df):
        df["comment"] = df["comment"].map(lambda text: self.__lemmatize_text(text))
        return df

In [ ]:


pipe =  Pipeline([
    ("ConvertData",ConvertData()),
    

    ("AddCurrencyFlag",AddCurrencyFlag()),
    ("AddSpamWordsFlag",AddSpamWordsFlag()),
    ("AddEmojiFlag",AddEmojiFlag()),
    ("AddContainFlag",AddContainFlag()),
    ("AddEmailFlag",AddEmailFlag()),
    ("AddPhoneFlag",AddPhoneFlag()),

    ("RemovePhoneLinkEmail",RemovePhoneLinkEmail()),
    ("RemoveStopWordsPunctuation",RemoveStopWordsPunctuation()),
    
    ("LemmatizeText",LemmatizeText()),

    ("AddLengthFlag",AddLengthFlag()),
    ("UpSample",UpSample())


])


In [ ]:
df = pipe.transform(df)
df.info()


In [ ]:
# import seaborn as sns
# sns.countplot(x="currency",data=df)
# sns.countplot(x="spam_word",data=df)
# sns.countplot(x="emoji",data=df)
# sns.countplot(x="contain",data=df)
# sns.countplot(x="email",data=df)
# sns.countplot(x="phone",data=df)
# sns.countplot(x="length",data=df)
sns.countplot(x="category",data=df)


In [ ]:
y = pd.DataFrame(df.category)
x = df.drop(["category"],axis=1)

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.8,test_size=0.2,random_state=0)

In [ ]:
X_train = {
    "Comment": tf.convert_to_tensor(x_train["comment"]),
    "Length": tf.convert_to_tensor(x_train["length"], dtype=tf.float32),
    "Currency": tf.convert_to_tensor(x_train["currency"], dtype=tf.float32),
    "Spam Words": tf.convert_to_tensor(x_train["spam_word"], dtype=tf.float32),
    "Emoji": tf.convert_to_tensor(x_train["emoji"], dtype=tf.float32),
    "Contain": tf.convert_to_tensor(x_train["contain"], dtype=tf.float32),
    "Email": tf.convert_to_tensor(x_train["email"], dtype=tf.float32),
    "Phone": tf.convert_to_tensor(x_train["phone"], dtype=tf.float32)
}

X_test={
    "Comment": tf.convert_to_tensor(x_test["comment"]),
    "Length": tf.convert_to_tensor(x_test["length"], dtype=tf.float32),
    "Currency": tf.convert_to_tensor(x_test["currency"], dtype=tf.float32),
    "Spam Words": tf.convert_to_tensor(x_test["spam_word"], dtype=tf.float32),
    "Emoji": tf.convert_to_tensor(x_test["emoji"], dtype=tf.float32),
    "Contain": tf.convert_to_tensor(x_test["contain"], dtype=tf.float32),
    "Email": tf.convert_to_tensor(x_test["email"], dtype=tf.float32),
    "Phone": tf.convert_to_tensor(x_test["phone"], dtype=tf.float32)
}

y_train = { "Spam" : tf.convert_to_tensor(y_train) }
y_test = { "Spam" : tf.convert_to_tensor(y_test) }


In [ ]:
string_input = tf.keras.layers.Input(shape=[], dtype=tf.string , name="Comment")
length_input   = tf.keras.layers.Input(shape=(1,),name="Length",dtype=tf.float32)
currency_input = tf.keras.layers.Input(shape=(1,),name="Currency",dtype=tf.float32)
spam_word_input = tf.keras.layers.Input(shape=(1,),name="Spam Words",dtype=tf.float32)
emoji_input = tf.keras.layers.Input(shape=(1,),name="Emoji",dtype=tf.float32)
contain_input = tf.keras.layers.Input(shape=(1,),name="Contain",dtype=tf.float32)
email_input = tf.keras.layers.Input(shape=(1,),name="Email",dtype=tf.float32)
phone_input = tf.keras.layers.Input(shape=(1,),name="Phone",dtype=tf.float32)

# offline file
file_or_url = "./models/nnlm/"

#online file
file_or_url = "https://tfhub.dev/google/nnlm-en-dim50/2"

hub_layer = hub.KerasLayer(file_or_url, dtype=tf.string, trainable=True,name="NNLM_Hub")

embedding_layer = hub_layer(string_input)

def CommonLayer(units:int,layer,name=None):    
    s1= tf.keras.layers.Dense(units=units,name = name,activation='relu')(layer)
    return s1

string_layer = CommonLayer(2500,embedding_layer)
string_layer = CommonLayer(2000,string_layer)
string_layer = CommonLayer(1500,string_layer)
string_layer = CommonLayer(1000,string_layer)
string_layer = CommonLayer(500,string_layer)
string_layer = tf.keras.layers.BatchNormalization()(string_layer)


length_layer = CommonLayer(256,length_input,"length_layer")
length_layer = CommonLayer(120,length_layer)


currency_layer = CommonLayer(256,currency_input,"currency_layer")
currency_layer = CommonLayer(62,currency_layer)

spam_word_layer = CommonLayer(256,spam_word_input,"spam_word_layer")
spam_word_layer = CommonLayer(62,spam_word_layer)

emoji_layer = CommonLayer(256,emoji_input,"emoji_layer")
emoji_layer = CommonLayer(62,emoji_layer)

contain_layer = CommonLayer(256,contain_input,"conatian_layer")
contain_layer = CommonLayer(256,contain_layer)

email_layer = CommonLayer(256,email_input,"email_layer")
email_layer = CommonLayer(54,email_layer)


phone_layer = CommonLayer(256,phone_input,"phone_layer")
phone_layer = CommonLayer(124,phone_layer)



concat_layer_level1 = tf.keras.layers.concatenate([length_layer,currency_layer,spam_word_layer])
concat_layer_level1 = CommonLayer(600,concat_layer_level1)


concat_layer_level2 = tf.keras.layers.concatenate([contain_layer,emoji_layer,email_layer,phone_layer])
concat_layer_level2 = CommonLayer(800,concat_layer_level2)


concat_layer_level = tf.keras.layers.concatenate([concat_layer_level1,concat_layer_level2])

sub_layer = CommonLayer(800,concat_layer_level,"features_layer")
sub_layer = tf.keras.layers.BatchNormalization()(sub_layer)


# Concatenate all input branches
concat_layer = tf.keras.layers.concatenate([string_layer,sub_layer ])
concat_layer  = tf.keras.layers.Dropout(rate=0.2)(concat_layer)

# Add dense and output layers
f1 = CommonLayer(1000,concat_layer)

f1 = CommonLayer(500,f1)
f1 = CommonLayer(250,f1)
f1 = CommonLayer(150,f1)
f1 = CommonLayer(64,f1)

output_layer = tf.keras.layers.Dense(1,activation='relu',name="category")(f1)

# Create the model
model = tf.keras.Model(inputs=[string_input, length_input,currency_input,spam_word_input,emoji_input,contain_input,email_input,phone_input], outputs=output_layer)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()
model.build([])
#67248

In [ ]:
# Plot model
keras.utils.plot_model(model, to_file='model.png',show_shapes=1,expand_nested=1,show_layer_activations=1)

# Display the image
# data = plt.imread('model.png')
# plt.imshow(data)

In [ ]:

# k = model.fit(X_train,
#           y_train,
#           epochs=1,
#           batch_size=12,
#           validation_data=(X_test, y_test),
#           callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)],
#           verbose=1
# )
model = tf.keras.models.load_model('../cyberadar_web/spam-model.h5', custom_objects={'KerasLayer':hub.KerasLayer})


In [ ]:
model.evaluate([X_test],y_test)

In [ ]:
# accuracy = k.history['accuracy']
# loss = k.history['loss']
# val_accuracy = k.history['val_accuracy']
# val_loss = k.history['val_loss']

# final_accuracy = accuracy[-1]
# final_loss = loss[-1]
# final_val_accuracy = val_accuracy[-1]
# final_val_loss = val_loss[-1]

# print("Final Accuracy:", final_accuracy)
# print("Final Loss:", final_loss)
# print("Final Validation Accuracy:", final_val_accuracy)
# print("Final Validation Loss:", final_val_loss)


In [ ]:
# model.evaluate([X_test],y_test)

In [ ]:
# accuracy = k.history['accuracy']
# loss = k.history['loss']
# val_accuracy = k.history['val_accuracy']
# val_loss = k.history['val_loss']

# # Plot the training and validation accuracy
# plt.plot(range(1, len(accuracy) + 1), accuracy, label='Training Accuracy')
# plt.plot(range(1, len(val_accuracy) + 1), val_accuracy, label='Validation Accuracy')
# plt.title('Training and Validation Accuracy')
# plt.xlabel('Epochs')
# plt.ylabel('Accuracy')
# plt.legend()
# plt.show()

# # Plot the training and validation loss
# plt.plot(range(1, len(loss) + 1), loss, label='Training Loss')
# plt.plot(range(1, len(val_loss) + 1), val_loss, label='Validation Loss')
# plt.title('Training and Validation Loss')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.legend()
# plt.show()
